# **AUTOMATIZACIÓN DE INGESTA DE DATOS NUEVOS**

### Importar la librería WATCHDOG

In [1]:
%pip install watchdog
%pip install pandas 
%pip install numpy 
%pip install SQLAlchemy 
%pip install pyodbc
%pip install nbformat

import pyodbc
import time
import pandas as pd
import numpy as np
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from sqlalchemy import create_engine

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.


In [4]:
class Watcher:
    # Directorio que se va a observar
    # Es necesario colocar la ruta del directorio local donde esén los archivos .csv quién ejecute el código
    DIRECTORY_TO_WATCH = "C:/Users/Usuario/Desktop/data/repo/Proyecto-Final-Henry/Data Engineer/csv"

    def __init__(self):
        # Inicialización del observador
        self.observer = Observer()

    def run(self):
        # Creación de un manejador de eventos
        event_handler = Handler()
        # Configuración del observador para que observe el directorio y subdirectorios
        self.observer.schedule(event_handler, self.DIRECTORY_TO_WATCH, recursive=True)
        # Inicio del observador
        self.observer.start()
        try:
            # Bucle infinito para mantener el programa en ejecución
            while True:
                # Pausa de 1 segundo para evitar un uso excesivo de la CPU
                time.sleep(1)
        except KeyboardInterrupt:
            # Detener el observador si se detecta una interrupción del teclado (Ctrl + C)
            self.observer.stop()
        # Esperar a que el observador termine antes de salir del programa
        self.observer.join()

class Handler(FileSystemEventHandler):
    @staticmethod
    def on_modified(event):
        # Verificar si el evento corresponde a un directorio
        if event.is_directory:
            return None
        # Verificar si el evento es una modificación de archivo
        elif event.event_type == 'modified':
            # Aquí puedes poner el código que quieres ejecutar:
            
            #-------------------------------------- CÓDIGO A EJECUTAR --------------------------------------#         
            
            # Cargar los CSV a los dfs con las columnas que vamos a ocupar
            df_team = pd.read_csv('csv/team.csv', usecols=['id', 'full_name','abbreviation','city', 'state'])
            df_team_details = pd.read_csv('csv/team_details.csv', usecols=['team_id','owner', 'generalmanager','headcoach', 'arena', 'arenacapacity'])
            df_team_history = pd.read_csv('csv/team_history.csv', usecols=['team_id', 'year_founded', 'year_active_till'])

            new_teams_data = {
                'team_id': [1610612738, 1610612739, 1610612752, 1610612740, 1610612753],
                'owner': ['Wyc Grousbeck, Steve Pagliuca, y otros socios', 'Dan Gilbert', 'James Dolan (Madison Square Garden Sports)', 'Gayle Benson', 'RDV Sports, Inc. (familia DeVos)'],
                'generalmanager': ['Brad Stevens', 'Mike Gansey', 'Scott Perry', 'Trajan Langdon', 'Anthony Parker'],
                'headcoach': ['Joe Mazzulla', 'J.B. Bickerstaff', 'Tom Thibodeau', 'Willie Green', 'Jamahl Mosley'],
                'arena': ['TD Garden', 'Rocket Mortgage FieldHouse', 'Madison Square Garden', 'Smoothie King Center', 'Amway Center'],
                'arenacapacity': [19156, 19432, 19812, 16867, 18846]
            }

            # Crear DataFrame con nuevos datos
            df_new_teams = pd.DataFrame(new_teams_data)

            # Concatenar DataFrames
            df_team_details = pd.concat([df_team_details, df_new_teams], ignore_index=True)

            # Renombrar la columna id a team_id
            df_team = df_team.rename(columns={'id': 'team_id'})

            # Fusionar las tablas df_team y df_team_details
            df_fusionado = pd.merge(df_team, df_team_details, on='team_id', how='outer')

            # Fusionar la tabla resultante anterior con df_team_history
            df_fusionado2 = pd.merge(df_fusionado, df_team_history, on='team_id', how='outer')

            teams = df_fusionado2.rename(columns={'team_id': 'id'})

            # Encontrar el valor máximo de la columna 'year_active_till'
            max_year_active_till = teams['year_active_till'].max()

            # Agregar la columna 'estado' basada en la condición
            teams['estado'] = np.where(teams['year_active_till'] < max_year_active_till, 0, 1)

            # Agregado de la tabla location al dataframe
            # Crear un nuevo DataFrame con las columnas "state" y "city"
            location = pd.DataFrame(teams, columns=['state','city'])

            # Eliminar filas duplicadas basadas en la columna "city"
            df_location_sin_duplicados = location.drop_duplicates(subset=['city'])

            # Crear un nuevo DataFrame con las columnas
            location = df_location_sin_duplicados

            # Generar una columna "ID" con valores únicos
            location['ID_location'] = range(1, len(location) + 1) 

            # Unir los DataFrames en función de las columnas "city" y "state"
            merge_location = pd.merge(teams, location, on=['city','state'], how='left')

            teams = merge_location

            teams = teams.drop(columns=['city','state'])

            ##Agregado de la tabla arena al dataframe
            # Crear un nuevo DataFrame con las columnas "arena" y "arenacapacity"
            arena = pd.DataFrame(teams, columns=['arena','arenacapacity'])

            # Eliminar filas duplicadas basadas en la columna "city"
            df_arena_sin_duplicados = arena.drop_duplicates(subset=['arena'])

            # Crear un nuevo DataFrame con las columnas
            arena = df_arena_sin_duplicados

            # Generar una columna "ID" con valores únicos
            arena['ID_arena'] = range(1, len(arena) + 1) 

            # Unir los DataFrames en función de las columnas "city" y "state"
            merge_arena = pd.merge(teams, arena, on=['arenacapacity','arena'], how='left')

            teams = merge_arena

            teams.rename(columns={"id": "ID_team"}, inplace=True)

            teams = teams.drop(columns=['arenacapacity','arena','year_founded','year_active_till','estado'])

            teams_History = df_team_history.rename(columns={'team_id': 'ID_team'})

            # Generar una columna "ID" con valores únicos
            teams_History['ID_history'] = range(1, len(teams_History) + 1)

            teams = teams.drop_duplicates(subset='ID_team')

            teams.fillna('Darko Rajaković', inplace=True)
            
            # Df city
            city_unique = df_team['city'].unique()  # Obtiene los valores únicos de la columna 'city'
            city = pd.DataFrame(city_unique, columns=['city'])  # Crea un DataFrame con los valores únicos

            city['ID_city'] = range(1, len(city) + 1)  #Creo el indice

            # Df state
            state_unique = df_team['state'].unique()  # Obtiene los valores únicos de la columna 'state'
            state = pd.DataFrame(state_unique, columns=['state'])  # Crea un DataFrame con los valores únicos

            state['ID_state'] = range(1, len(state) + 1)  #Creo el indice

            #Megere
            # Unir los DataFrames en función de las columnas "city" y "state"
            merge_city = pd.merge(city, location, on=['city'], how='left')
            merge_total = pd.merge(state, merge_city, on=['state'], how='left')

            merge_total = merge_total.drop(columns=['state','city'])

            #Borro columnas de location
            location = merge_total
            
            # Leer los archivos CSV
            df_draft_combine_stats = pd.read_csv('csv/draft_combine_stats.csv')
            df_draft_history = pd.read_csv('csv/draft_history.csv')

            # Seleccion de las columnas a utilizar
            columnas_draft_combine_stats  = ['player_id','weight','wingspan','standing_reach','body_fat_pct','standing_vertical_leap','max_vertical_leap','lane_agility_time','modified_lane_agility_time','three_quarter_sprint','bench_press']
            df_draft_combine_stats_reducido = df_draft_combine_stats.loc[:,columnas_draft_combine_stats]

            columnas_draft_history  = ['person_id','season','player_name','round_number','round_pick','overall_pick','team_id','organization','organization_type']
            df_draft_history_reducido = df_draft_history.loc[:,columnas_draft_history]

            df_draft_history_reducido.rename(columns={"person_id": "player_id"}, inplace=True)

            # DataFrame Draft
            draft = pd.merge(df_draft_combine_stats_reducido, df_draft_history_reducido, on='player_id', how='outer')

            # Df proveniencia
            # Crear un nuevo DataFrame con las columnas "organization" y "organization_type"
            proveniencia = pd.DataFrame(draft, columns=['organization','organization_type'])  

            # Eliminar filas duplicadas basadas en la columna "organization"
            df_sin_duplicados = proveniencia.drop_duplicates(subset=["organization"])

            # Crear un nuevo DataFrame con las columnas 
            proveniencia = df_sin_duplicados

            # Generar una columna "ID" con valores únicos
            proveniencia["ID_proveniencia"] = range(1, len(proveniencia) + 1)

            # Unir los DataFrames en función de las columnas "organization" y "organization_type"
            merge = pd.merge(draft, proveniencia, on=["organization", "organization_type"], how="left")

            draft = merge

            draft = draft.drop(columns=['organization','organization_type'])

            draft = draft.rename(columns={'player_id':'ID_player', 'team_id':'ID_team'})

            draft = draft[draft['ID_team']>=1610612737]
            
            # Leer el archivo CSV
            df_common_player_info = pd.read_csv('csv/common_player_info.csv')
            df_inactive_players = pd.read_csv('csv/inactive_players.csv')
            df_player = pd.read_csv('csv/player.csv')

            #DF Players
            player = df_common_player_info.drop(columns=['display_first_last','school','country','display_last_comma_first','display_fi_last','player_slug','last_affiliation','rosterstatus','games_played_current_season_flag','team_name','team_abbreviation','team_code','team_city','playercode','from_year','to_year','dleague_flag','nba_flag','games_played_flag','draft_year','draft_round','draft_number','greatest_75_flag'])
            # Renombrar la columna id a person_id
            df_player = df_player.rename(columns={'id': 'person_id'})

            # Fusionar las tablas df_player y player
            player = pd.merge(player, df_player, on='person_id', how='inner')
            player = player.drop(columns=['last_name_y','first_name_y','full_name'])

            # Renombrar la columna id a ID_player
            player = player.rename(columns={'person_id': 'ID_player', 'first_name_x':'first_name', 'last_name_x':'last_name', 'team_id':'ID_team'})

            player = player[player['ID_team']>=1610612737]
            
            # Cargar los archivos CSV
            df_game = pd.read_csv('csv/game.csv')  
            df_game_info = pd.read_csv('csv/game_info.csv')  
            df_game_summary = pd.read_csv('csv/game_summary.csv')  
            df_line_score = pd.read_csv('csv/line_score.csv')

            # Unir las tablas
            df_combine = df_game.merge(df_game_info, on='game_id', how='inner')

            df_combine_summary = df_combine.merge(df_game_summary, on='game_id', how='inner')

            df_combine_total = df_combine_summary.merge(df_line_score, on='game_id', how='inner')

            # Seleccionar las columnas

            game_stats = df_combine_total[['game_id','pts_home_x', 'pts_qtr1_home', 'pts_qtr2_home', 'pts_qtr3_home', 'pts_qtr4_home', 
                                            'pts_qtr1_away', 'pts_qtr2_away', 'pts_qtr3_away', 'pts_qtr4_away', 'pts_away_x', 
                                            'attendance', 'game_time', 'natl_tv_broadcaster_abbreviation', 'live_period_time_bcast']]

            game_stats['ID_game_stats'] = range(1, len(game_stats) + 1)  # Creación del id


            df_games = df_combine_total[['game_id', 'game_date_x', 'team_id_home_x', 'wl_home', 'team_id_away_x', 'wl_away', 'season_id', 'season', 'season_type']]

            # Renombrar la columna team_id_home a ID_team en df_games
            df_games = df_games.rename(columns={'team_id_home_x': 'ID_team'})

            games = pd.merge(df_games, teams[['ID_team', 'ID_location']], on='ID_team', how='outer')


            # Renombrar la columna game_id a ID_game
            games = games.rename(columns={'game_id': 'ID_game', 'game_date_x': 'game_date', 'team_id_away_x': 'team_id_away'})


            # Creación de tabla de Season 
            df_season = df_combine_total[['season_id', 'season', 'season_type']]
            season = df_season.drop_duplicates(subset=['season_id'])
            season = season.rename(columns={'season_id': 'ID_season'})
            season = season.drop_duplicates(subset='ID_season')

            games = games.rename(columns={'season_id': 'ID_season'})
            games = games.drop(columns=['season','season_type'])
            games = games.drop_duplicates(subset='ID_game')

            game_stats = game_stats.rename(columns={'game_id': 'ID_game'})

            # ID_season en tabla de Draft
            draft = pd.merge(draft, season[['ID_season', 'season']], on='season', how='inner')
            draft_d = draft.drop(columns=['season'])
            draft = draft_d

            draft['ID_draft'] = range(1, len(draft) + 1)  # Creación del id

            games = games[games['ID_team']>=1610612737]


            # Arreglo de errores

            prueba = pd.merge(player,draft[['ID_player','player_name']],on='ID_player',how='outer')
            prueba = prueba.drop_duplicates('ID_player')
            prueba_2 = prueba.dropna(subset=['player_name'])
            player = prueba_2.drop(columns=['first_name','last_name'])

            game_stats = game_stats.drop_duplicates(subset='ID_game')

            game_stats = pd.merge(game_stats, games['ID_game'], on='ID_game', how='inner')
            games = games.dropna(subset=['ID_game'])
            
            from sqlalchemy import create_engine
            import pyodbc

            # Parámetros de la conexión
            server = 'localhost\\SQLEXPRESS'  # Reemplaza con el nombre de tu servidor
            database = 'DAFT01_Grupo1_nba'  # Reemplaza con el nombre de tu base de datos

            # Crear la cadena de conexión
            conn_str = (
                'DRIVER={ODBC Driver 17 for SQL Server};'
                'SERVER=' + server + ';'  # Reemplaza con el nombre de tu servidor
                'DATABASE=' + database + ';'  # La base de datos a la que te quieres conectar
                'Trusted_Connection=yes;'
            )

            # Crear la conexión
            conn = pyodbc.connect(conn_str, autocommit=True)  # El autocommit en True permite crear la base desde el código

            # Crear un cursor
            cursor = conn.cursor()

            # Season
            df_viejo_season = pd.read_sql_query(f"SELECT * FROM season", conn)
            merged_season = season.merge(df_viejo_season, on='ID_season', how='left', suffixes=('', '_old'), indicator=True)
            season_nuevo = merged_season[merged_season['_merge'] == 'left_only'].drop(columns=[col for col in merged_season.columns if col.endswith('_old')] + ['_merge'])

            # State
            df_viejo_State = pd.read_sql_query(f"SELECT * FROM State", conn)
            merged_State = state.merge(df_viejo_State, on='ID_state', how='left', suffixes=('', '_old'), indicator=True)
            State_nuevo = merged_State[merged_State['_merge'] == 'left_only'].drop(columns=[col for col in merged_State.columns if col.endswith('_old')] + ['_merge'])

            # City
            df_viejo_City = pd.read_sql_query(f"SELECT * FROM City", conn)
            merged_City = city.merge(df_viejo_City, on='ID_city', how='left', suffixes=('', '_old'), indicator=True)
            City_nuevo = merged_City[merged_City['_merge'] == 'left_only'].drop(columns=[col for col in merged_City.columns if col.endswith('_old')] + ['_merge'])

            # Arena
            df_viejo_Arena = pd.read_sql_query(f"SELECT * FROM Arena", conn)
            merged_Arena = arena.merge(df_viejo_Arena, on='ID_arena', how='left', suffixes=('', '_old'), indicator=True)
            Arena_nuevo = merged_Arena[merged_Arena['_merge'] == 'left_only'].drop(columns=[col for col in merged_Arena.columns if col.endswith('_old')] + ['_merge'])

            # Location
            df_viejo_Location = pd.read_sql_query(f"SELECT * FROM Location", conn)
            merged_Location = location.merge(df_viejo_Location, on='ID_location', how='left', suffixes=('', '_old'), indicator=True)
            Location_nuevo = merged_Location[merged_Location['_merge'] == 'left_only'].drop(columns=[col for col in merged_Location.columns if col.endswith('_old')] + ['_merge'])

            # Proveniencia
            df_viejo_Proveniencia = pd.read_sql_query(f"SELECT * FROM Proveniencia", conn)
            merged_Proveniencia = proveniencia.merge(df_viejo_Proveniencia, on='ID_proveniencia', how='left', suffixes=('', '_old'), indicator=True)
            Proveniencia_nuevo = merged_Proveniencia[merged_Proveniencia['_merge'] == 'left_only'].drop(columns=[col for col in merged_Proveniencia.columns if col.endswith('_old')] + ['_merge'])

            # teams
            df_viejo_teams = pd.read_sql_query(f"SELECT * FROM teams", conn)
            merged_teams = teams.merge(df_viejo_teams, on='ID_team', how='left', suffixes=('', '_old'), indicator=True)
            teams_nuevo = merged_teams[merged_teams['_merge'] == 'left_only'].drop(columns=[col for col in merged_teams.columns if col.endswith('_old')] + ['_merge'])

            # teams_History
            df_viejo_teams_History = pd.read_sql_query(f"SELECT * FROM teams_History", conn)
            merged_teams_History = teams_History.merge(df_viejo_teams_History, on='ID_history', how='left', suffixes=('', '_old'), indicator=True)
            teams_History_nuevo = merged_teams_History[merged_teams_History['_merge'] == 'left_only'].drop(columns=[col for col in merged_teams_History.columns if col.endswith('_old')] + ['_merge'])

            # Games
            df_viejo_Games = pd.read_sql_query(f"SELECT * FROM Games", conn)
            merged_Games = games.merge(df_viejo_Games, on='ID_game', how='left', suffixes=('', '_old'), indicator=True)
            Games_nuevo = merged_Games[merged_Games['_merge'] == 'left_only'].drop(columns=[col for col in merged_Games.columns if col.endswith('_old')] + ['_merge'])

            # Game_stats
            df_viejo_Game_stats = pd.read_sql_query(f"SELECT * FROM Game_stats", conn)
            merged_Game_stats = game_stats.merge(df_viejo_Game_stats, on='ID_game_stats', how='left', suffixes=('', '_old'), indicator=True)
            Game_stats_nuevo = merged_Game_stats[merged_Game_stats['_merge'] == 'left_only'].drop(columns=[col for col in merged_Game_stats.columns if col.endswith('_old')] + ['_merge'])

            # Player
            df_viejo_Player = pd.read_sql_query(f"SELECT * FROM Player", conn)
            merged_Player = player.merge(df_viejo_Player, on='ID_player', how='left', suffixes=('', '_old'), indicator=True)
            Player_nuevo = merged_Player[merged_Player['_merge'] == 'left_only'].drop(columns=[col for col in merged_Player.columns if col.endswith('_old')] + ['_merge'])

            # Draft
            df_viejo_Draft = pd.read_sql_query(f"SELECT * FROM Draft", conn)
            merged_Draft = draft.merge(df_viejo_Draft, on='ID_draft', how='left', suffixes=('', '_old'), indicator=True)
            Draft_nuevo = merged_Draft[merged_Draft['_merge'] == 'left_only'].drop(columns=[col for col in merged_Draft.columns if col.endswith('_old')] + ['_merge'])

            # Close the cursor and the connection
            cursor.close()
            conn.close()

            # Parámetros de la conexión
            server = 'localhost\\SQLEXPRESS'  # Reemplaza con el nombre de tu servidor
            database = 'DAFT01_Grupo1_nba'  # Reemplaza con el nombre de tu base de datos

            # Crear la cadena de conexión utilizando la autenticación de Windows
            connection_string = f'mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes'

            # Crear el motor de conexión
            engine = create_engine(connection_string)

            season_nuevo.to_sql('season', engine, if_exists='append', index=False)
            State_nuevo.to_sql('State', engine, if_exists='append', index=False)
            City_nuevo.to_sql('City', engine, if_exists='append', index=False)
            Arena_nuevo.to_sql('Arena', engine, if_exists='append', index=False)
            Location_nuevo.to_sql('Location', engine, if_exists='append', index=False)
            Proveniencia_nuevo.to_sql('Proveniencia', engine, if_exists='append', index=False)
            teams_nuevo.to_sql('teams', engine, if_exists='append', index=False)
            teams_History_nuevo.to_sql('teams_History', engine, if_exists='append', index=False)
            Games_nuevo.to_sql('Games', engine, if_exists='append', index=False)
            Game_stats_nuevo.to_sql('Game_stats', engine, if_exists='append', index=False)
            Player_nuevo.to_sql('Player', engine, if_exists='append', index=False)
            Draft_nuevo.to_sql('Draft', engine, if_exists='append', index=False)

            #-------------------------------------- FIN DEL CÓDIGO --------------------------------------# 

            print(f'Archivo modificado: {event.src_path}')

if __name__ == '__main__':
    # Crear una instancia de la clase Watcher y ejecutar el método run()
    w = Watcher()
    w.run()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_23940\3755356686.py:228: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_stats['ID_game_stats'] = range(1, len(game_stats) + 1)  # Creación del id
C:\Users\Usuario\AppData\Local\Temp\ipykernel_23940\3755356686.py:299: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_viejo_season = pd.read_sql_query(f"SELECT * FROM season", conn)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_23940\3755356686.py:304: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objec

Archivo modificado: C:/Users/Usuario/Desktop/data/repo/Proyecto-Final-Henry/Data Engineer/csv\team.csv
